# Analysis of available Data

Import required libraries

In [1]:
%pip install -r requirements.txt

ERROR: Could not find a version that satisfies the requirement zipfile (from versions: none)
ERROR: No matching distribution found for zipfile


In [2]:
import os
import numpy as np
from constant import *
from data_func import *
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import keras.layers as layers
import tensorflow as tf



In [3]:
os.chdir(rootPath)

In [4]:
raw_data,raw_labels,filenames, SAMPLE_RATE = load_files()
# print('raw data shape is ---',raw_data.shape)
import warnings

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

Extraction Finished.
Data loaded
Data shape :  (119,)


c:\Users\as116\OneDrive - Indian Institute of Technology Bombay\Documents B\Sem 7\IE643\IE643_180260004_CHALLENGE\data_func.py:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.array(data)


In [5]:
from audiomentations import Compose, AddGaussianNoise, TimeStretch, TimeMask, Shift,TimeStretch, Trim, Normalize
# import numpy as np


augment = Compose([
    Trim(top_db=20,p=1),
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
    TimeStretch(min_rate=0.8, max_rate=1.25, p=0.5),
    Shift(min_fraction=-0.5, max_fraction=0.5, p=0.5)
])

In [6]:
data_folds=[]
num_folds=50
for i in range(num_folds):
    data=[]
    for sound in raw_data:
        data.append(augment(sound,SAMPLE_RATE))
    data_folds.append(data)

data_folds = np.array(data_folds)
# labels = np.array(raw_labels)
print('data shape---',data_folds.shape)
print('labels shape',len(raw_labels))    

data shape--- (50, 119)
labels shape 119


In [7]:
data=[]
labels=[]
for fold  in data_folds:
    for i,j in enumerate(fold):
        data.append(j)
        labels.append(raw_labels[i])

# data = np.array(data, dtype=np.float64)
# labels = np.array(labels, dtype=np.float64)

In [8]:
stft_data = []
for i, sound in enumerate(data):
    stft = np.abs(librosa.stft(sound, n_fft=512, hop_length=256, win_length=512))
    stft = np.mean(stft, axis=1)
    stft_data.append(stft)

stft_data = np.array(stft_data)


In [9]:
num_labels = len(labels_dir)
from sklearn.preprocessing import LabelBinarizer
one_hot = LabelBinarizer().fit(range(1,num_labels+1))
one_hot_labels = one_hot.transform(labels)

In [10]:
from sklearn.model_selection import train_test_split as split

data_train, data_test, labels_train, labels_test = split(
    stft_data, one_hot_labels, train_size=0.6, shuffle=True, random_state=42
)
data_val, data_test, labels_val, labels_test = split(
    data_test, labels_test, train_size=0.5, shuffle=True, random_state=42
)


In [11]:
print("data train shape", data_train.shape)
print(
    "label train {}".format(type(labels_train)),
    len(labels_train) if type(labels_train) == "list" else labels_train.shape,
)

data train shape (3570, 257)
label train <class 'numpy.ndarray'> (3570, 7)


In [12]:
#some error in trying to train the model using these datasets, couldn't figure out

# train_dataset = tf.data.Dataset.from_tensor_slices((data_train,labels_train))
# val_dataset = tf.data.Dataset.from_tensor_slices((data_val,labels_val))

In [13]:
model = keras.models.Sequential()
model.add(layers.Dense(256, activation="relu", input_shape=(input_features,)))

model.add(layers.Dense(256, activation="relu"))

model.add(layers.Dense(128, activation="relu"))

model.add(layers.Dense(128, activation="relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_labels, activation="relu"))

model.compile(
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
    optimizer="adam",
)


In [14]:
print('Final datset shapes are')
print('data train shape',data_train.shape)
print('labels train shape',labels_train.shape)

Final datset shapes are
data train shape (3570, 257)
labels train shape (3570, 7)


In [15]:
model.fit(data_train,labels_train,batch_size=8,epochs=20,validation_data=(data_val,labels_val))

Epoch 1/20
447/447 [==============================] - 6s 8ms/step - loss: 1.0507 - accuracy: 0.6465 - val_loss: 0.6965 - val_accuracy: 0.7807
Epoch 2/20
447/447 [==============================] - 3s 7ms/step - loss: 0.3705 - accuracy: 0.8885 - val_loss: 0.2822 - val_accuracy: 0.9025
Epoch 3/20
447/447 [==============================] - 4s 9ms/step - loss: 0.2235 - accuracy: 0.9317 - val_loss: 0.4077 - val_accuracy: 0.9034
Epoch 4/20
447/447 [==============================] - 3s 6ms/step - loss: 0.2243 - accuracy: 0.9398 - val_loss: 0.1111 - val_accuracy: 0.9672
Epoch 5/20
447/447 [==============================] - 4s 8ms/step - loss: 0.1206 - accuracy: 0.9667 - val_loss: 0.2326 - val_accuracy: 0.9176
Epoch 6/20
447/447 [==============================] - 3s 7ms/step - loss: 0.0861 - accuracy: 0.9754 - val_loss: 0.0444 - val_accuracy: 0.9882
Epoch 7/20
447/447 [==============================] - 3s 6ms/step - loss: 0.0692 - accuracy: 0.9818 - val_loss: 0.1342 - val_accuracy: 0.9790
Epoch 

In [16]:
model.evaluate(data_test,labels_test)

38/38 [==============================] - 0s 6ms/step - loss: 4.0663e-04 - accuracy: 1.0000


[0.0004066294350195676, 1.0]